To run this notebook provide the following:
- path to the training dataset;
- path to store the results (text files, containing model settings, loss function values and
  average dice scores vs epochs);

In [ ]:
import torch.nn as nn
import torch.optim
import torch.nn.functional as F
import torch.utils.data
from datetime import date

from db_load_pytable import LoadPyTable
from model import UNet
from augmentation import *
from dice_generalization import *

In [ ]:
## Provide path to the training dataset;
path_to_trainset = '/home/chepburn/MRI_Bone/Jupyter/ARTICLE_I/Databases/data_7-5-21'

## Provide path to save trained model parameters;
path_to_store = '/home/chepburn/MRI_Bone/Jupyter/ARTICLE_I/Results/test/'

In [ ]:
## -------------------------------- Specify hyper-parameters --------------------------------
## ---------------- Model architecture;
depth = 4
filter_number = 4
kernel_size = 3
padding = 1

## ---------------- Data augmentation;
## Specify number of augmentation steps to perform within one epoch;
steps = 350
    
## ---------------- Optimizer parameters;
optimizer = 'Adam' 
batchsize = 4
lr = 0.001 ## learning rate;
epochs = 100

## ---------------- Outputs;
## Loss function value will be outputted every 10th epoch;
output_number = 10

In [ ]:
## ---------------- Specify transformation parameters (data augmentation) --------------------

## Create Transform class object; pass a dictionary with transformation parameters;
## Choose device_mode "cuda" to allocate the output on GPU;
ranges = {}
ranges['degree'] = 10
ranges['scale_yx'] = (0.9,1.1)
ranges['shear_yx'] = 0.2
ranges['sigma_points'] = (3,12)

transform = Transform()
transform.warp_mode = 'reflect'
transform.device_mode = 'cuda' 
transform.ranges = ranges

## ----------------------------------- Evaluation metric ---------------------------------
## Create Dice class object to compute average dice score at each epoch;
dice = Dice()

In [ ]:
## ------------------------- Save current settings to a text file --------------------------
file = open(path_to_store + 'settings.txt', 'w')

comments = '''
'''
settings_description = '''Comments: %s

Model:
    Model depth: %s;
    Filter number: %s;
    Convolutional kernel size: %s;
    Padding: %s;

Parameter Initialization: default initialization; (see pytorch documentation at
//pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d);
    
Adam optimizer settings: 
    Batchsize: %s;
    Learing rate: %s;
    Epochs: %s;

During each training epoch %s augmentation steps are performed;

Data augmentation. Transformation parameters: 
ranges['degree'] = 10
ranges['scale_yx'] = (0.9,1.1)
ranges['shear_yx'] = 0.2
ranges['sigma_points'] = (3,12)
transform.warp_mode = 'reflect'

Data is taken from: %s
'''%(comments, depth, filter_number, kernel_size, padding, batchsize, lr, epochs, steps,
     path_to_trainset)

file.write(settings_description)
file.close()

In [ ]:
## Load training dataset; pass pytable to torch dataloader;
## Specify batchsize and whether to shuffle the data;
trainset = LoadPyTable(path_to_trainset)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batchsize, shuffle = True)

In [ ]:
## Define the model;
model = UNet(depth = depth,
             filter_number = filter_number,
             kernel_size = kernel_size,
             padding = padding)

## Allocate model on GPU BEFORE constructing the optimizer;
model = model.cuda() 
    
## Construct optimizer (Adam);
optim = torch.optim.Adam(model.parameters(), lr = lr)

## Define loss function (BCE);
loss_fn = nn.BCELoss()

## Create lists to store loss function values, mean dice scores;
trainloss, train_mean_scores = [], []

In [ ]:
#### ---------------------------------------------------------------------- Loop over epochs;
## Note: model is in the training mode by default;
for k in range(epochs):
                    
    ## Set an initial value of the loss funtion (to be accumulated over augmentation steps);
    run_loss = 0
    
    ## Create a list to store dice score for each step, averaged over classes and samples in
    ## the batch;
    trainbatch_scores = []
    
    #### -------------------------------------------------------- Loop over augmentation steps; 
    for step in range(steps):

        ## NB: Set gradients to zero not to accumulate them over the training epochs;
        optim.zero_grad()

        ## Create a list: [ [image batch 1, rs batch 1], ..., [image batch n, rs batch n] ];
        ## (rs - reference standard);
        datalist = list(trainloader)
        
        ## Generate a random integer from the range (0, number of batches);
        rn = np.random.randint(low = 0, high = len(datalist))

        ## Choose a random sample from the list: [image batch, rs batch];
        ## Output: two 4D torch tensors of size [N,1,H,W]; (requires_grad = False);
        ## (N - batch size);
        sample = datalist[rn]
        inputs, targets = sample[0], sample[1]

        ## ---------------------------- Data Augmentation ------------------------------

        ## Pass size of the input tensor to the Transform class object;
        ## (size of the last batch may differ);
        transform.dim = inputs.shape

        ## Transform image, target batch (elastic, affine, random flip, intensity augmentation);;
        ## Output: two 4D torch tensors of size [N,1,H,W], allocated on GPU;
        t_inputs, t_targets = transform.batch_transform(inputs, targets)

        ## -----------------------------------------------------------------------------

        ## Pass transformed inputs to the model; 
        ## Output: 4D torch tensor [N,1,H,W];
        ## (requires_grad = True => computational graph will be stored to compute gradients);
        prediction = model(t_inputs)

        ## Compute loss function value, given model prediction and transformed targets;
        ## Output: 1D torch tensor;
        loss = loss_fn(prediction, t_targets)

        ## Increment run loss variable;
        ## (use .item() method to retrieve float value from the output tensor);
        run_loss += loss.item()

        ## Compute the loss function gradients;
        loss.backward()

        ## Update model parameters;
        optim.step()

        ## ------------------------------- Compute dice score -----------------------------
        ## Don't store the computational graph, unnecessary here;
        with torch.no_grad():

            ## Inputs: two 4D torch tensors of size [N,1,H,W], output: float;
            trainbatch_sd = dice.compute_average_dice(prediction, t_targets)
            trainbatch_scores.append(trainbatch_sd) ## append to the list;
        ## -------------------------------------------------------------------------------
    #### -------------------------------------------------------------- Loop over steps ends;
    
    ## Save loss function value accumulated over steps;
    trainloss.append(run_loss)
    
    ## Save dice scores, averaged over classes, samples in the batch and augmentation steps;
    ## Convert list to numpy array to be able to compute mean;
    trainbatch_scores = np.asarray(trainbatch_scores)
    train_mean_scores.append( np.mean(trainbatch_scores) )

    ## Output loss function value each XXth epoch; 
    if k % output_number == 0:

        print('Loss at {}th iteration, training set: {}'.format(k,round(trainloss[k],2)))
    
    ## Save model parameters for the last 10 models;
    if k > 90:
        
        ## Specify model name;
        model_name = 'model_'+ str(date.today()) + '_epoch_'+ str(k) + '.pt'
        torch.save( model.state_dict(), path_to_store + model_name)
#### ----------------------------------------------------------------- Loop over epochs ends;

In [ ]:
## Convert lists to numpy arrays;
trainloss = np.asarray(trainloss)
train_mean_scores = np.asarray(train_mean_scores)

## Save loss function value and average dice scores to a txt file;
header = '''
Col.0: Loss function value
Col.1: Average 2D soft dice
'''
np.savetxt(path_to_store + 'training_details', np.c_[trainloss, train_mean_scores],
           header = header)